# グラフ彩色問題のヒューリスティック手法

- seq_assignment: 与えられた点の番号の順に，彩色可能な最小の番号の色で塗っていく．
- largest_first: 点の次数（隣接する点の数）が大きいものから順に彩色する．
    - 次数が大きい方から使う理由は？
- dsatur: 構築法の途中で得た動的な情報をもとに，次に彩色する点を選択する．具体的には，次に彩色する点を， 隣接する点ですでに使われた色数が最大のもの（同点の場合には，彩色されていない隣接点の数が最大のもの）とする．
- recursive_largest_fit: 同じ色を塗ることができる点の集合（色クラス）は，互いに隣接していてはいけない． これは，安定集合に他ならない． 新しい色で彩色するための安定集合を順次求めることによって解を構築する．
    - 以下を繰り返す．次数(塗られていない隣接頂点数)が最も大きい頂点に新しい色を塗り，その頂点を含む大きな安定集合を貪欲に求める (次数が大きい順に？なぜ？)

In [1]:
import graphtools as gts
from gcp_heur import seq_assignment, largest_first, dsatur, recursive_largest_fit
from gcp_heur import recursive_largest_fit_2

print("*** graph coloring problem ***")
print()

print("instance randomly created")
nodes, adj = gts.rnd_adj_fast(100, 0.5)

print("sequential assignment")
color, K = seq_assignment(nodes, adj)
print("solution: z =", K)
print(color)
print()
print("largest fit")
color, K = largest_first(nodes, adj)
print("solution: z =", K)
print(color)
print()
print("dsatur")
color, K = dsatur(nodes, adj)
print("solution: z =", K)
print(color)
print()
print("recursive largest fit")
color, K = recursive_largest_fit(nodes, adj)
print("solution: z =", K)
print(color)
print()
print("recursive largest fit (minimum ver)")
color, K = recursive_largest_fit_2(nodes, adj)
print("solution: z =", K)
print(color)
print()


*** graph coloring problem ***

instance randomly created
sequential assignment
solution: z = 22
[0, 0, 1, 2, 1, 3, 2, 4, 5, 1, 0, 6, 4, 2, 3, 6, 5, 1, 7, 6, 3, 8, 5, 4, 8, 0, 7, 3, 7, 9, 8, 0, 2, 1, 9, 10, 9, 0, 10, 11, 11, 3, 8, 8, 12, 5, 6, 13, 4, 9, 12, 13, 2, 5, 6, 13, 14, 10, 5, 3, 13, 13, 11, 15, 12, 7, 16, 17, 17, 2, 9, 7, 17, 14, 16, 1, 12, 15, 18, 8, 9, 19, 4, 20, 9, 10, 18, 6, 11, 15, 14, 21, 12, 19, 16, 4, 10, 18, 18, 10]

largest fit
solution: z = 21
[15, 11, 9, 2, 3, 13, 10, 2, 5, 17, 3, 1, 14, 14, 9, 15, 10, 4, 13, 12, 20, 4, 8, 18, 6, 19, 15, 13, 3, 16, 6, 18, 14, 19, 12, 1, 5, 15, 7, 3, 0, 4, 2, 17, 17, 19, 12, 2, 11, 8, 0, 10, 14, 6, 20, 3, 4, 4, 0, 12, 7, 7, 5, 1, 10, 16, 15, 7, 6, 9, 10, 16, 7, 9, 11, 17, 18, 1, 6, 8, 1, 5, 8, 3, 1, 14, 13, 2, 2, 1, 9, 8, 16, 5, 4, 14, 0, 13, 13, 11]

dsatur
solution: z = 19
[6, 10, 11, 2, 5, 9, 15, 2, 14, 10, 3, 1, 13, 2, 3, 12, 11, 3, 9, 12, 12, 3, 5, 3, 6, 6, 7, 8, 7, 17, 14, 2, 3, 14, 0, 1, 0, 10, 10, 6, 5, 3, 4, 11, 0, 14, 15, 